In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name = "BAAI/bge-small-en-v1.5")
Settings.llm = None
Settings.chunk_size = 1024
Settings.chunk_overlap = 64

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [ ]:
!mkdir lectures
!cp /content/drive/MyDrive/ML/datasets/mth101/* /content/lectures

In [ ]:
documents = SimpleDirectoryReader("lectures").load_data()

In [ ]:
print(len(documents))

86


In [ ]:
#store vectors into vector database
index = VectorStoreIndex.from_documents(documents)

In [ ]:
#set number of docs to retrieve
top_k = 3

#configure retriever
retriever = VectorIndexRetriever(index = index, similarity_top_k = top_k)

In [ ]:
#assemble query engine
query_engine = RetrieverQueryEngine(
    retriever = retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)]
)

In [ ]:
#load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=False, revision = "main")

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

adapter_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

In [ ]:
from auto_gptq import exllama_set_max_input_length
model = exllama_set_max_input_length(model, max_input_length=2800)

In [ ]:
#load tokenizer
tokenizer =AutoTokenizer.from_pretrained(model_name, use_fast = True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
from urllib.parse import quote_from_bytes
query = "What is the second derivative test?"
response = query_engine.query(query)

#query documents
context = "Context: \n"
for i in range(top_k):
  context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context: 
1
Lecture 29 : Mixed Derivative Theorem, MVT and Extended MVT
Iff:R2→R, then fxis a function from R2toR(if it exists). So one can analyze the existence of
fxx= (fx)x=∂2f
∂x2=∂
∂x(∂f
∂x) and fxy= (fx)y=∂2f
∂y∂x=∂
∂y(∂f
∂x)
which are partial derivatives of fxwith respect xoryand, similarly the existence of fyyandfyx.
These are called second order partial derivatives of f.
The following example shows that, in general, fxyneed not be equal to fyx.
Example : Letf(x, y) =xyx2−y2
x2+y2if (x, y)̸= (0,0) and f(0,0) = 0. Note that
fy(h,0) = lim
k→0f(h, k)−f(h,0)
k= lim
k→0h(h2−k2)
h2+k2=h
and
fyx(0,0) = lim
h→0fy(h,0)−fy(0,0)
h= 1.
Similarly, fxy(0,0) =−1.
Theorem 29.1(Mixed derivative theorem) : Iff(x, y)and its partial derivatives fx, fy, fxy
andfyxare deﬁned in a neighborhood of (x0, y0)and all are continuous at (x0, y0)thenfxy(x0, y0) =
fyx(x0, y0).
We will not present the proof of this result here. The proof is given in the text book.
Mean Value Theorem : We will present the MVT f

In [ ]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]MTH-GPT, functioning as a virtual mathematician, with an expertise in Real Analysis and related domains \
will help provide a detailed solution to the user's query with reference to the context provided.

{context}
Give solution the following query. Use the context above.

{comment}
[/INST]
"""

In [ ]:
comment = "What is the second derivative test?"
prompt = prompt_template_w_context(context, comment)

print(prompt)

[INST]ShawGPT, functioning as a virtual mathematician, with an expertise in Real Analysis and related domains will help provide a detailed solution to the user's query with reference to the context provided.

Context: 
1
Lecture 29 : Mixed Derivative Theorem, MVT and Extended MVT
Iff:R2→R, then fxis a function from R2toR(if it exists). So one can analyze the existence of
fxx= (fx)x=∂2f
∂x2=∂
∂x(∂f
∂x) and fxy= (fx)y=∂2f
∂y∂x=∂
∂y(∂f
∂x)
which are partial derivatives of fxwith respect xoryand, similarly the existence of fyyandfyx.
These are called second order partial derivatives of f.
The following example shows that, in general, fxyneed not be equal to fyx.
Example : Letf(x, y) =xyx2−y2
x2+y2if (x, y)̸= (0,0) and f(0,0) = 0. Note that
fy(h,0) = lim
k→0f(h, k)−f(h,0)
k= lim
k→0h(h2−k2)
h2+k2=h
and
fyx(0,0) = lim
h→0fy(h,0)−fy(0,0)
h= 1.
Similarly, fxy(0,0) =−1.
Theorem 29.1(Mixed derivative theorem) : Iff(x, y)and its partial derivatives fx, fy, fxy
andfyxare deﬁned in a neighborhood o

In [ ]:
print(len(prompt))

5749


In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
print(len(inputs))

2


In [ ]:
print(len(inputs['input_ids'][0]))

2672


In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]ShawGPT, functioning as a virtual mathematician, with an expertise in Real Analysis and related domains will help provide a detailed solution to the user's query with reference to the context provided.

Context: 
1
Lecture 29 : Mixed Derivative Theorem, MVT and Extended MVT
Iff:R2→R, then fxis a function from R2toR(if it exists). So one can analyze the existence of
fxx= (fx)x=∂2f
∂x2=∂
∂x(∂f
∂x) and fxy= (fx)y=∂2f
∂y∂x=∂
∂y(∂f
∂x)
which are partial derivatives of fxwith respect xoryand, similarly the existence of fyyandfyx.
These are called second order partial derivatives of f.
The following example shows that, in general, fxyneed not be equal to fyx.
Example : Letf(x, y) =xyx2−y2
x2+y2if (x, y)̸= (0,0) and f(0,0) = 0. Note that
fy(h,0) = lim
k→0f(h, k)−f(h,0)
k= lim
k→0h(h2−k2)
h2+k2=h
and
fyx(0,0) = lim
h→0fy(h,0)−fy(0,0)
h= 1.
Similarly, fxy(0,0) =−1.
Theorem 29.1(Mixed derivative theorem) : Iff(x, y)and its partial derivatives fx, fy, fxy
andfyxare deﬁned in a neighborho

In [ ]:
query = '''Let S be the part of the cylinder z = 1 − x2, 0 ≤ x ≤ 1, −2 ≤ y ≤ 2. Let C be the
boundary curve of the surface S. Let F (x, y, z) = yi + yj + zk. Find the unit outer normal to
S, curl F and evaluate ∮
C F · dR where C is oriented counterclockwise as viewed from above the
surface.'''

response = query_engine.query(query)

#query documents
context = "Context: \n"
for i in range(top_k):
  context = context + response.source_nodes[i].text + "\n\n"

prompt = prompt_template_w_context(context, query)

print(prompt)

[INST]ShawGPT, functioning as a virtual mathematician, with an expertise in Real Analysis and related domains will help provide a detailed solution to the user's query with reference to the context provided.

Context: 
The above problem can be done directly by calculating the line integral. In that
case we have to parametrize the boundary which consists of four smooth curves. One has to be
careful with the direction in each piece. The calculation also becomes lengthy.
2. We can also consider the surface given in the previous problem as a level surface deﬁned by
f(x, y, z ) =z−1 +x2= 0. In this case an unit normal is n=∇f
∥∇f∥=2xi+k√
1+4x2.

2
is considered to be the orientation w.r.to n.
In practice, we get the orientation of the boundary curve (w.r.to a given n) using the right-hand
rule: If we wrap around the surface with the right hand by pointing the thumb in the direction of
the normal then roughly the ﬁngers of the right hand are pointing towards the orientation of the
curve. One

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]ShawGPT, functioning as a virtual mathematician, with an expertise in Real Analysis and related domains will help provide a detailed solution to the user's query with reference to the context provided.

Context: 
The above problem can be done directly by calculating the line integral. In that
case we have to parametrize the boundary which consists of four smooth curves. One has to be
careful with the direction in each piece. The calculation also becomes lengthy.
2. We can also consider the surface given in the previous problem as a level surface deﬁned by
f(x, y, z ) =z−1 +x2= 0. In this case an unit normal is n=∇f
∥∇f∥=2xi+k√
1+4x2.

2
is considered to be the orientation w.r.to n.
In practice, we get the orientation of the boundary curve (w.r.to a given n) using the right-hand
rule: If we wrap around the surface with the right hand by pointing the thumb in the direction of
the normal then roughly the ﬁngers of the right hand are pointing towards the orientation of the
curve.

In [ ]:
def prompter(comment):
  #query documents
  response = query_engine.query(comment)

  context = "Context: \n"

  for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

  prompt = prompt_template_w_context(context, comment)

  return prompt

In [ ]:
query = "Prove that if f is continuous on [a, b] then f is integrable."
prompt = prompter(query)

[INST]ShawGPT, functioning as a virtual mathematician, with an expertise in Real Analysis and related domains will help provide a detailed solution to the user's query with reference to the context provided.

Context: 
2
The second F.T.C. explains why the indeﬁnite integral of F′is deﬁned to be F.
Remark : The proof of Theorem 17.2 becomes simpler if, instead of assuming fto be integrable,
we make stonger assumption that fis continuous on [ a, b]. In fact, the proof follows from Theorem
17.1 (prove !).
Problem 1: Letpbe a ﬁxed number and let fbe a continuous function on Rthat satisﬁes the
equation f(x+p) =f(x)for every x∈R. Show that the integral∫a+p
af(t)dthas the same value
for every real number a.
Solution : Suppose a, p > 0. Then by the ﬁrst F.T.C., we have
d
da(∫a+p
af(t)dt) =d
da(∫a+p
0f(t)dt−∫a
0f(t)dt) =f(a+p)−f(a) = 0 .
Problem 2: Letfbe a continuous function on [0, π/2]and∫π/2
0f(t)dt= 0. Show that there exists
ac∈(0, π/2)such that f(c) = 2 cos2c.
Solution : Deﬁne Fon [0,π
2]

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=1024)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]ShawGPT, functioning as a virtual mathematician, with an expertise in Real Analysis and related domains will help provide a detailed solution to the user's query with reference to the context provided.

Context: 
2
The second F.T.C. explains why the indeﬁnite integral of F′is deﬁned to be F.
Remark : The proof of Theorem 17.2 becomes simpler if, instead of assuming fto be integrable,
we make stonger assumption that fis continuous on [ a, b]. In fact, the proof follows from Theorem
17.1 (prove !).
Problem 1: Letpbe a ﬁxed number and let fbe a continuous function on Rthat satisﬁes the
equation f(x+p) =f(x)for every x∈R. Show that the integral∫a+p
af(t)dthas the same value
for every real number a.
Solution : Suppose a, p > 0. Then by the ﬁrst F.T.C., we have
d
da(∫a+p
af(t)dt) =d
da(∫a+p
0f(t)dt−∫a
0f(t)dt) =f(a+p)−f(a) = 0 .
Problem 2: Letfbe a continuous function on [0, π/2]and∫π/2
0f(t)dt= 0. Show that there exists
ac∈(0, π/2)such that f(c) = 2 cos2c.
Solution : Deﬁne Fon [0,

In [ ]:
def mthgpt(input_query):
  input_prompt = prompter(input_query)

  inputs = tokenizer(input_prompt, return_tensors="pt")

  outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                           attention_mask=inputs["attention_mask"].to("cuda"),
                           pad_token_id=tokenizer.eos_token_id,
                           max_new_tokens=1024)

  result = tokenizer.batch_decode(outputs)[0]

  solution = result.split("[/INST]\n")[1].strip().replace("</s>", "\n~MTHgpt")
  print(input_query + "\n" + solution)

In [ ]:
query = "Prove that if f is continuous on [a, b] then f is integrable."
mthgpt(query)

Prove that if f is continuous on [a, b] then f is integrable.
To prove that a continuous function is integrable, we will use the Riemann Criterion for integrability. This criterion states that a function is integrable if and only if for every ε > 0, there exists a partition P of [a, b] such that the Riemann sum S(P, f) - L(P, f) < ε.

Let ε > 0. Since f is continuous on [a, b], it is uniformly continuous. Therefore, there exists δ > 0 such that |s - t| < δ implies |f(s) - f(t)| < ε.

Now let P be a partition of [a, b] such that ∆ xi < δ for all i. Then,

Mi - mi ≤ |f(xi) - f(xi - 1)| * ∆ xi < ε * ∆ xi.

Thus,

U(P, f) - L(P, f) = ∑ i (Mi - mi) * ∆ xi < ε * (b - a).

Since ε and b - a are arbitrary, this shows that f is integrable.
~MTHgpt
